In [1]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings("ignore")

In [2]:
tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = "2020-01-01"
end_date = "2024-12-31"

stock_data = yf.download(
    tickers, 
    start=start_date, 
    end=end_date
)['Adj Close']

#From this point on all can stay the same if you were to input ur own portolfio with its returns above. 
#Therefore you can hedge ur shrape ratio for other strategies 


port_returns = (
    stock_data
    .pct_change()
    .sum(axis=1)
)

port_returns.name = "port_returns"
cumulative_ret = (port_returns + 1).cumprod()
port_returns

[*********************100%***********************]  3 of 3 completed


Date
2020-01-02 00:00:00+00:00    0.000000
2020-01-03 00:00:00+00:00   -0.027405
2020-01-06 00:00:00+00:00    0.037207
2020-01-07 00:00:00+00:00   -0.015752
2020-01-08 00:00:00+00:00    0.039132
                               ...   
2024-12-23 00:00:00+00:00    0.016795
2024-12-24 00:00:00+00:00    0.028456
2024-12-26 00:00:00+00:00   -0.002202
2024-12-27 00:00:00+00:00   -0.045063
2024-12-30 00:00:00+00:00   -0.034388
Name: port_returns, Length: 1257, dtype: float64

In [3]:
fama_french = web.DataReader(
    "F-F_Research_Data_Factors_daily",
    "famafrench",
    start_date,
    end_date
)[0]

fama_french = fama_french / 100  # Convert to decimals
fama_french.index = fama_french.index.tz_localize("UTC")

data = fama_french.join(port_returns, how="inner")

excess_returns = data.port_returns - data.RF


# Prepare X variables
X = data[["SMB", "HML"]]
X = sm.add_constant(X)

# Rolling regression setup
window_size = 100  # One year of daily data

rolling_params = []
for start in range(len(data) - window_size + 1):
    end = start + window_size
    X_window = X.iloc[start:end]
    y_window = excess_returns.iloc[start:end]
    
    model = sm.OLS(y_window, X_window).fit()
    rolling_params.append(model.params)

rolling_params_df = pd.DataFrame(rolling_params, index=data.index[window_size-1:], columns=X.columns)

# Unhedged returns remain constant for the entire dataset
unhedged_returns = port_returns

# Calculate hedge portfolio for rolling regression
rolling_hedge_weights = -rolling_params_df[["SMB", "HML"]]

# Align indices for hedge portfolio
rolling_hedge_portfolio = (
    data[["SMB", "HML"]]
    .loc[rolling_hedge_weights.index]
    .values * rolling_hedge_weights.values
).sum(axis=1)

rolling_hedge_portfolio = pd.Series(
    rolling_hedge_portfolio,
    index=rolling_hedge_weights.index,
    name="hedge_portfolio"
)

# Adjust portfolio returns with rolling hedge
rolling_hedged_portfolio_returns = unhedged_returns.loc[rolling_hedge_portfolio.index] + rolling_hedge_portfolio

# Create a DataFrame for comparison
hedge = pd.DataFrame({  # Use the full unhedged returns directly
    "hedged_returns": rolling_hedged_portfolio_returns
})

# Ensure consistent index for comparisons
hedge = hedge.loc[rolling_hedged_portfolio_returns.index]

# Performance metrics
sharpe = hedge.mean() / hedge.std()
print(sharpe)

hedged_returns    0.078856
dtype: float64


In [4]:
#For regression
X = data[["SMB", "HML"]]

X = sm.add_constant(X)

model = sm.OLS(excess_returns, X).fit()

hedge_weights = -model.params[1:]

hedge_portfolio = (data[["SMB", "HML"]] @ hedge_weights).dropna()

hedged_portfolio_returns = port_returns.loc[hedge_portfolio.index] + hedge_portfolio

hedge = pd.DataFrame({
    "unhedged_returns": port_returns.loc[hedged_portfolio_returns.index],
    "hedged_returns": hedged_portfolio_returns
})

hedge.mean() / hedge.std()

unhedged_returns    0.057647
hedged_returns      0.061138
dtype: float64